In [1]:
%pip install -qU langchain flashrank langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from index_pipeline import vector_store
from rich import print

reranker = FlashrankRerank(top_n=3)
vector_retriever = vector_store.as_retriever(search_kwargs={"k": 10})

retriever = ContextualCompressionRetriever(base_compressor=reranker, base_retriever=vector_retriever)
retriever_result = retriever.invoke("why does self-attention have an advantage over recurrent layers in terms of parallelization and path length for long-range dependencies?")

print(retriever_result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[
    Document(
        metadata={
            'id': 4,
            'relevance_score': np.float32(0.9995491),
            'page': 1,
            'title': '',
            'author': '',
            'source': 
'/Users/loki/Documents/work/Personal/learnings/rag-cookbook-langchain/techniques/retriever/aiaun.pdf',
            'creator': 'LaTeX with hyperref',
            'moddate': '2024-04-10T21:11:43+00:00',
            'subject': '',
            'trapped': '/False',
            'keywords': '',
            'producer': 'pdfTeX-1.40.25',
            'page_label': '2',
            'total_pages': 15,
            'creationdate': '2024-04-10T21:11:43+00:00',
            'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 
6.3.5'
        },
        page_content='1 Introduction\nRecurrent neural networks, long short-term memory [13] and gated recurrent 
[7] neural networks\nin particular, have been firmly established as state of the art approaches in sequence 
modeling and\ntransduction problems such as language modeling and machine translation [ 35, 2, 5]. 
Numerous\nefforts have since continued to push the boundaries of recurrent language models and 
encoder-decoder\narchitectures [38, 24, 15]. Recurrent models typically factor computation along the symbol 
positions of the input and output\nsequences. Aligning the positions to steps in computation time, they generate a 
sequence of hidden\nstates ht, as a function of the previous hidden state ht−1 and the input for position t. This 
inherently\nsequential nature precludes parallelization within training examples, which becomes critical at 
longer\nsequence lengths, as memory constraints limit batching across examples. Recent work has 
achieved\nsignificant improvements in computational efficiency through factorization tricks [21] and 
conditional\ncomputation [32], while also improving model performance in case of the latter. The 
fundamental\nconstraint of sequential computation, however, remains. Attention mechanisms have become an integral 
part of compelling sequence modeling and transduc-\ntion models in various tasks, allowing modeling of dependencies
without regard to their distance in\nthe input or output sequences [2, 19]. In all but a few cases [27], however, 
such attention mechanisms\nare used in conjunction with a recurrent network. In this work we propose the 
Transformer, a model architecture eschewing recurrence and instead\nrelying entirely on an attention mechanism to 
draw global dependencies between input and output. The Transformer allows for significantly more parallelization 
and can reach a new state of the art in\ntranslation quality after being trained for as little as twelve hours on 
eight P100 GPUs. 2 Background\nThe goal of reducing sequential computation also forms the foundation of the 
Extended Neural GPU\n[16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic 
building\nblock, computing hidden representations in parallel for all input and output positions. In these 
models,\nthe number of operations required to relate signals from two arbitrary input or output positions grows\nin
the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes\nit more difficult
to learn dependencies between distant positions [ 12]. In the Transformer this is\nreduced to a constant number of 
operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an 
effect we counteract with Multi-Head Attention as\ndescribed in section 3.2. Self-attention, sometimes called 
intra-attention is an attention mechanism relating different positions\nof a single sequence in order to compute a 
representation of the sequence. Self-attention has been\nused successfully in a variety of tasks including reading 
comprehension, abstractive summarization,\ntextual entailment and learning task-independent sentence 
representat

In [11]:
from langchain_openai import ChatOpenAI
from rich import print

chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
response = chat_model.invoke(f"You are an expert in the field of AI and you are given a question and a document. You need to answer the question based on the document. Here is the question: Why does self-attention have an advantage over recurrent layers in terms of parallelization and path length for long-range dependencies? Here is the document: {retriever_result}")

print(response.content)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Self-attention has an advantage over recurrent layers in terms of parallelization and path length for long-range 
dependencies due to its architectural design. Specifically, self-attention allows for all positions in a sequence 
to be connected with a constant number of sequentially executed operations, which means that it can process inputs 
in parallel. In contrast, recurrent layers require a linear number of sequential operations (O(n)), which limits 
their ability to parallelize computations, especially as sequence lengths increase.

Additionally, the path length for learning long-range dependencies is significantly shorter in self-attention 
mechanisms. In self-attention, the maximum path length between any two input and output positions is constant 
(O(1)), making it easier to learn dependencies regardless of their distance in the sequence. On the other hand, 
recurrent layers have a maximum path length that grows linearly with the sequence length (O(n)), which complicates 
the learning of long-range dependencies.

In summary, self-attention's ability to connect all positions with constant operations and its constant path length
for dependencies provide significant advantages over recurrent layers, particularly in terms of computational 
efficiency and the ability to learn long-range relationships in data.